# Interest Rate Prediction

## Case Study

This data belongs to a loan aggregator agency which connects loan applications to different financial institutions in attempt to get the best interest rate. They want to now utilise past data to predict interest rate given by any financial institute just by looking at loan application characteristics.

To achieve that , they have decided to do a POC with a data from a particular financial institution. The data is given in the file "loans data.csv". Lets begin: 

## Step 0: Basic Imports

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Step 1: Load dataset

In [2]:
train_file='data/loan_data_train.csv' #relative path. So in present working directory it will look for data folder.
test_file='data/loan_data_test.csv'

df_train=pd.read_csv(train_file) #read_csv is a function in pandas.
df_test=pd.read_csv(test_file)               

In [5]:
print(df_train.shape) #check no of rows (ie the no of data points) and cols (ie the no of features)
df_train.head(2) #display the first 5 columns in your dataset

(2200, 15)


,ID,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
0,79542.0,25000,25000,18.49%,60 months,debt_consolidation,27.56%,VA,MORTGAGE,8606.56,720-724,11,15210,3.0,5 years
1,75473.0,19750,19750,17.27%,60 months,debt_consolidation,13.39%,NY,MORTGAGE,6737.50,710-714,14,19070,3.0,4 years


14 features and 1 target variable

In [6]:
#test data does not have interest rate
print(df_test.shape)
df_test.sample(2)

(300, 14)


,ID,Amount.Requested,Amount.Funded.By.Investors,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
64,16158,10000,5334.01,36 months,car,8.83%,TX,RENT,4166.67,730-734,5,6381,0,1 year
154,42148,10000,9950,36 months,debt_consolidation,12.63%,CA,RENT,7166.67,680-684,11,10309,1,< 1 year


## Step 2: Data Visualisation and Feature Selection

### 2.1. Check dtypes

In [8]:
df_train.dtypes #this will return a list of the columns and their respective dtypes

ID                                float64
Amount.Requested                   object
Amount.Funded.By.Investors         object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

In [7]:
#df_train.dtypes #this will return a list of the columns and their respective dtypes
df_train.dtypes.unique() #will give the unique dtypes in my dataset

array([dtype('float64'), dtype('O')], dtype=object)

In [9]:
#df_train.select_dtypes("object") --> this will return a dataframe with only object type columns
obj_cols = df_train.select_dtypes("object").columns
obj_cols, len(obj_cols)

(Index(['Amount.Requested', 'Amount.Funded.By.Investors', 'Interest.Rate',
        'Loan.Length', 'Loan.Purpose', 'Debt.To.Income.Ratio', 'State',
        'Home.Ownership', 'FICO.Range', 'Open.CREDIT.Lines',
        'Revolving.CREDIT.Balance', 'Employment.Length'],
       dtype='object'), 12)

Dtypes for 12 feature has to be fixed

In [10]:
print(df_train[obj_cols].shape)
df_train[obj_cols].sample(5) #slicing of dataframe-->dataframe[list_of_desired_cols]

(2200, 12)


,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Employment.Length
395,35000,34950,8.90%,36 months,debt_consolidation,20.46%,CT,RENT,735-739,19,21536,1 year
1194,15000,15000,6.03%,36 months,debt_consolidation,8.72%,MA,MORTGAGE,800-804,17,13151,2 years
1475,10000,10000,15.31%,36 months,debt_consolidation,14.61%,CA,RENT,680-684,18,13051,5 years
886,3000,3000,7.14%,36 months,debt_consolidation,12.85%,PA,RENT,730-734,8,3210,5 years
14,7350,7350,10.74%,60 months,debt_consolidation,29.82%,WA,MORTGAGE,745-749,15,15093,4 years


In [11]:
df_train["Loan.Length"].value_counts() #unique value and the count of each value

36 months    1722
60 months     476
.               1
Name: Loan.Length, dtype: int64

In [12]:
df_train["Loan.Purpose"].value_counts()

debt_consolidation    1147
credit_card            394
other                  174
home_improvement       135
major_purchase          84
small_business          80
car                     45
wedding                 35
medical                 26
moving                  25
house                   19
vacation                18
educational             14
renewable_energy         3
Name: Loan.Purpose, dtype: int64

In [13]:
df_train["Home.Ownership"].value_counts()

MORTGAGE    1018
RENT         999
OWN          177
OTHER          4
NONE           1
Name: Home.Ownership, dtype: int64

In [14]:
df_train["Employment.Length"].value_counts()

10+ years    575
< 1 year     229
2 years      217
3 years      203
5 years      181
4 years      162
1 year       159
6 years      134
7 years      109
8 years       95
9 years       66
.              1
Name: Employment.Length, dtype: int64

### 2.1.1. Some observations

1. 'Amount.Requested': **convert it to numeric**
2. 'Amount.Funded.By.Investors': **drop**
3. 'Interest.Rate': **remove % and then to numeric**
4. 'Loan.Length': **dummies for categories**
5. 'Loan.Purpose': **dummies for categories**
6. 'Debt.To.Income.Ratio': **remove % and then to numeric**
7. 'State': **dummies for categories**
8. 'Home.Ownership': **dummies for categories**
9. 'FICO.Range': **replace it by a numeric column which is average of the range**
10. 'Open.CREDIT.Lines': **convert it to numeric**
11. 'Revolving.CREDIT.Balance': **convert it to numeric**
12. 'Employment.Length': **convert it to number**


##### Lets group by operation:

1. drop: 
    - ID 
    - Amount.Funded.By.Investors
<br>
2. convert it to numeric 
    - Amount.Requested
    - Open.CREDIT.Lines
    - Revolving.CREDIT.Balance
<br>
3. remove % and then to numeric
    - Interest Rate
    - Debt to income ratio
<br>
4. replace it by a numeric column which is average of the range
    - FICO.Range 
<br>
5. convert to number:
    - Employment Length 
<br>
6. dummies for categories with good occurence rate:
    - Loan Lenth
    - Loan Purpose
    - State
    - Home ownership

### 2.1.2. Fix Dtypes

#### Operation 1: df.drop([ ], inplace=True)

In [15]:
df_train.shape

(2200, 15)

In [16]:
df_train.drop(['ID','Amount.Funded.By.Investors'], axis=1, inplace=True)
#with inplace=True df_train will itself change. 

In [17]:
df_train.shape

(2200, 13)

#### Operation 2: pd.to_numeric

We can see that many columns which should have really been numbers have been imported as character columns , probably because some characters values in those columns in the files. We'll convert all such columns to numbers .

In [18]:
df_train.dtypes

Amount.Requested                   object
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                  object
Revolving.CREDIT.Balance           object
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

In [19]:
for col in ['Amount.Requested', 'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
    df_train[col]=pd.to_numeric(df_train[col], errors='coerce') 

In [20]:
df_train.dtypes

Amount.Requested                  float64
Interest.Rate                      object
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio               object
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

#### Operation 3: First String operation df[col].str.string_operation --> pd.to_numeric

variable `Interest.Rate` and `Debt.To.Income.Ratio` contain "%" sign in their values and because of which they have come as character columns in the data. Lets remove these percentages first.

In [23]:
#"aa%bb".replace("%","") #"aabb"

In [21]:
df_train['Interest.Rate'].sample(2)

1026    22.78%
2026    17.77%
Name: Interest.Rate, dtype: object

In [24]:
for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    print(col)
    df_train[col]=df_train[col].str.replace("%","") #first convert the row value to string type
                                                    #and then perfrom replace operation

Interest.Rate
Debt.To.Income.Ratio


In [25]:
#Lets check the effect of replace operation
df_train['Interest.Rate'].sample(2)

1499    21.48
1561    13.11
Name: Interest.Rate, dtype: object

In [26]:
#Now we can convert to numeric
for col in ['Interest.Rate','Debt.To.Income.Ratio']:
    df_train[col]=pd.to_numeric(df_train[col],errors='coerce') 

In [27]:
df_train.dtypes

Amount.Requested                  float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
FICO.Range                         object
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
dtype: object

#### Operation 4: .str.split(expand=True) -->df_train[new_name]

If we look at first few values of variable FICO.Range , we can see that we can convert it to numeric by taking average of the range given. To do that first we need to split the column with "-", so that we can have both end of ranges in separate columns and then we can simply average them.

In [28]:
"660-770".split("-") #for understanding

['660', '770']

In [29]:
df_train['FICO.Range'].sample(2)

918    760-764
355    670-674
Name: FICO.Range, dtype: object

In [30]:
k=df_train['FICO.Range'].str.split("-", expand=True).astype(float)

In [31]:
#now k is a new dataframe with two columns and same no of rows
k.head()

,0,1
0,720.0,724.0
1,710.0,714.0
2,690.0,694.0
3,710.0,714.0
4,730.0,734.0


In [32]:
k.dtypes

0    float64
1    float64
dtype: object

In [33]:
#creating a new column "fico" with average from k[0] (ie lower range) and k[1] (ie upper range)
df_train['fico']=0.5*(k[0]+k[1])
del df_train['FICO.Range']

In [35]:
df_train.shape

(2200, 13)

In [34]:
df_train.dtypes

Amount.Requested                  float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                  object
fico                              float64
dtype: object

#### Operation 5: np.where(condition, value_if_True, value_if_False)

In [36]:
#checking unique values for "Employment.Length" and their corresponding values
df_train['Employment.Length'].value_counts()

10+ years    575
< 1 year     229
2 years      217
3 years      203
5 years      181
4 years      162
1 year       159
6 years      134
7 years      109
8 years       95
9 years       66
.              1
Name: Employment.Length, dtype: int64

In [37]:
df_train['Employment.Length']=df_train['Employment.Length'].str.replace('years',"")
df_train['Employment.Length']=df_train['Employment.Length'].str.replace('year',"")

In [38]:
#verifying the two replace operations we performed
df_train['Employment.Length'].value_counts()

10+     575
< 1     229
2       217
3       203
5       181
4       162
1       159
6       134
7       109
8        95
9        66
.         1
Name: Employment.Length, dtype: int64

In [39]:
"78+00"[:2] #take index 0 and index 1. 

'78'

In [40]:
#np.where(condition, value_if_True, value_if_False)
df_train['Employment.Length'] = np.where(df_train['Employment.Length'].str[:2]=="10",10,
                                         df_train['Employment.Length'])

df_train['Employment.Length']=np.where(df_train['Employment.Length'].str[0]=="<",0,
                                       df_train['Employment.Length'])

In [41]:
df_train['Employment.Length'].value_counts()

10    575
0     229
2     217
3     203
5     181
4     162
1     159
6     134
7     109
8      95
9      66
.       1
Name: Employment.Length, dtype: int64

In [42]:
df_train['Employment.Length']=pd.to_numeric(df_train['Employment.Length'],errors='coerce')

In [ ]:
df_train['Employment.Length'].value_counts()

In [43]:
df_train.dtypes

Amount.Requested                  float64
Interest.Rate                     float64
Loan.Length                        object
Loan.Purpose                       object
Debt.To.Income.Ratio              float64
State                              object
Home.Ownership                     object
Monthly.Income                    float64
Open.CREDIT.Lines                 float64
Revolving.CREDIT.Balance          float64
Inquiries.in.the.Last.6.Months    float64
Employment.Length                 float64
fico                              float64
dtype: object

#### Operation 6: One hot encoding

In [44]:
# Notice that to apply string function on pandas data frame columns you need to str attribute
cat_cols=df_train.select_dtypes(['object']).columns
cat_cols

Index(['Loan.Length', 'Loan.Purpose', 'State', 'Home.Ownership'], dtype='object')

In [45]:
print("*"*50)
for col in cat_cols:
    print(col)
    print("-"*50)
    print(df_train[col].nunique())
    print("-"*50)
    print(df_train[col].value_counts())
    print("*"*50)

**************************************************
Loan.Length
--------------------------------------------------
3
--------------------------------------------------
36 months    1722
60 months     476
.               1
Name: Loan.Length, dtype: int64
**************************************************
Loan.Purpose
--------------------------------------------------
14
--------------------------------------------------
debt_consolidation    1147
credit_card            394
other                  174
home_improvement       135
major_purchase          84
small_business          80
car                     45
wedding                 35
medical                 26
moving                  25
house                   19
vacation                18
educational             14
renewable_energy         3
Name: Loan.Purpose, dtype: int64
**************************************************
State
--------------------------------------------------
46
--------------------------------------------------
CA   

In [47]:
# you can use following method if you want to ignore categories with too low frequencies ,
# in next section for logistic regression we will be using  pandas' get dummies function. 
# you can work with either of these . 
# ignoring categories with low frequencies however will result in fewer columns without 
# affecting model performance too much .
to_store = []
new_cols = []

for col in cat_cols:
    
    freqs = df_train[col].value_counts()
    categories = freqs.index[freqs>20]
    to_store.append((col, categories))
    
    for category in categories:
        name=col+'.'+category
        new_cols.append(name)
        df_train[name]=(df_train[col]==category).astype(int)
    del df_train[col]
    print(col)

KeyError: 'Loan.Length'

In [48]:
df_train.shape

(2200, 50)

In [ ]:
new_cols #we are storing all categorical feature names so that extracting numerical feature name is easy. 

In [ ]:
to_store #we are storing "features" used for one-hot-encoding, and for each feature the corresponding categories
         #used. This will help in replicating the same procedure for the test dataset. 

In [49]:
df_train.dtypes.unique() #mission successful

array([dtype('float64'), dtype('int64')], dtype=object)

In [50]:
df_train.dtypes

Amount.Requested                   float64
Interest.Rate                      float64
Debt.To.Income.Ratio               float64
Monthly.Income                     float64
Open.CREDIT.Lines                  float64
Revolving.CREDIT.Balance           float64
Inquiries.in.the.Last.6.Months     float64
Employment.Length                  float64
fico                               float64
Loan.Length.36 months                int64
Loan.Length.60 months                int64
Loan.Purpose.debt_consolidation      int64
Loan.Purpose.credit_card             int64
Loan.Purpose.other                   int64
Loan.Purpose.home_improvement        int64
Loan.Purpose.major_purchase          int64
Loan.Purpose.small_business          int64
Loan.Purpose.car                     int64
Loan.Purpose.wedding                 int64
Loan.Purpose.medical                 int64
Loan.Purpose.moving                  int64
State.CA                             int64
State.NY                             int64
State.FL   

### 2.2. Correlation Coefficient

In [ ]:
df_train.columns.drop(new_cols) #list of all numerical features

In [ ]:
numerical = df_train[df_train.columns.drop(new_cols)]
numerical.shape

In [ ]:
ax, fig = plt.subplots(figsize=(10,8))
sns.heatmap(numerical.corr(), annot=True, cmap='coolwarm')
plt.show()

### 2.3. Outliers

In [ ]:
from scipy import stats

fig, axs = plt.subplots(ncols=3, nrows=3, figsize=(30, 25))
index = 0
axs = axs.flatten()
for k,v in numerical.items():
    sns.boxplot(y=k, data=numerical, ax=axs[index])
    index += 1
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
for k, v in numerical.items(): #k=column_name, v=values
    q1 = v.quantile(0.25)
    q3 = v.quantile(0.75)
    irq = q3 - q1
    v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
    perc = np.shape(v_col)[0] * 100.0 / np.shape(numerical)[0]
    print("Column %s outliers = %.2f%%" % (k, perc))

### 2.4. Missing values: SimpleImputer

In [ ]:
df_train.isnull().sum()

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [ ]:
# imputer.fit(df_train) #learn the positions which has to be imputed
# imputer.transform(df_train) #actually perform the imputer operation 
# imputer.fit(df_train) + imputer.transform(df_train) = imputer.fit_transform(df_train)

In [ ]:
#return type for imputer is always array. We have to convert that back to dataframe
# How to convert array to dataframe:
# new_df = pd.Dataframe(array)

In [ ]:
df_train = pd.DataFrame(imputer.fit_transform(df_train), columns=df_train.columns)

In [ ]:
df_train.isnull().sum()

## Step 3: Defining Training and Test Set

In [ ]:
#dataframe.values --> will return array
X_train, y_train = df_train.loc[:,df_train.columns!="Interest.Rate"].values, df_train["Interest.Rate"].values
X_train.shape, y_train.shape

## Step 4: Data Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler #mean=0, std_deviation=1.0
std = StandardScaler().fit(X_train)
X_train = std.transform(X_train)

#### There are three different types of scaler: Standard, MixMax and Robust. Read about them here: http://benalexkeen.com/feature-scaling-with-scikit-learn/

## Step 5: Test set pipeline

In [ ]:
def preprocessing(dataframe, to_store, std):
    
    #Operation1
    dataframe = dataframe.drop(['ID','Amount.Funded.By.Investors'],axis=1)
    
    #Operation2
    for col in ['Amount.Requested', 'Open.CREDIT.Lines','Revolving.CREDIT.Balance']:
        dataframe[col]=pd.to_numeric(dataframe[col],errors='coerce')
    
    #Operation3
    for col in ['Debt.To.Income.Ratio']:
        dataframe[col] = dataframe[col].str.replace("%","") 
    
    for col in ['Debt.To.Income.Ratio']:
        dataframe[col]=pd.to_numeric(dataframe[col],errors='coerce') 
    
    #Operation4
    k=dataframe['FICO.Range'].str.split("-",expand=True).astype(float)
    dataframe['fico']=0.5*(k[0]+k[1])
    del dataframe['FICO.Range']
    
    #Operation5
    dataframe['Employment.Length']=dataframe['Employment.Length'].str.replace('years',"")
    dataframe['Employment.Length']=dataframe['Employment.Length'].str.replace('year',"")
    
    dataframe['Employment.Length']=np.where(dataframe['Employment.Length'].str[:2]=="10",10,
                                       dataframe['Employment.Length'])

    dataframe['Employment.Length']=np.where(dataframe['Employment.Length'].str[0]=="<",0,
                                       dataframe['Employment.Length'])
    
    dataframe['Employment.Length']=pd.to_numeric(dataframe['Employment.Length'],errors='coerce')
    
    #Operation6
    for column, categories in to_store:
        for category in categories:
            name=column+'.'+category
            dataframe[name]=(dataframe[column]==category).astype(int)
        del dataframe[column]
    
    #Missing values
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    dataframe = pd.DataFrame(imputer.fit_transform(dataframe), columns=dataframe.columns)
    
    #data scaling
    X_test = dataframe.values
    X_test = std.transform(X_test) #standard scaling
    
    return X_test

In [ ]:
X_test = preprocessing(df_test, to_store, std)
X_test.shape

## Step 6: Modelling

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
lm=LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
test_pred = lm.predict(X_test)

We can write these to a csv file for submission like this :

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)

### Ridge  Regression

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [ ]:
model=Ridge()

In [ ]:
alpha=np.linspace(1,100,100) #defiing range for values for lambda
params={'alpha':alpha}

In [ ]:
#GridSearchCV by default choses best model based on max score obtained. 
grid_search = GridSearchCV(model, param_grid=params, cv=10, scoring='neg_mean_absolute_error')

In [ ]:
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_estimator_

In [ ]:
grid_search.cv_results_

 if you want you can now fit a ridge regression model with obtained value of alpha , although there is no need, grid search automatically fits the best estimator on the entire data, you can directly use this to make predictions on test_data. But if you want to look at coefficients , its much more convenient to fit the model with direct function

Using the report function given below you can see the cv performance of top few models as well, that will the tentative performance

In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
report(grid_search.cv_results_,100)

In [ ]:
test_pred=grid_search.predict(X_test)

In [ ]:
pd.DataFrame(test_pred).to_csv("mysubmission.csv",index=False)